# Run MPNN interface design on the binders

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/projects/crispy_shifty/projects/crispy_crispies
running on node: dig30


### Set working directory to the root of the crispy_shifty repo

In [2]:
os.chdir("/projects/crispy_shifty")

### Run MPNN on the interfaces

In [4]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "00_mpnn_binders"
design_list_file = "/home/pleung/projects/peptide_binders/r0/peptide_binders/06_analyze_sequencing/to_redesign.list"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")
options = " ".join(
    [
        "out:level 200",
    ]
)
extra_kwargs = {"mpnn_design_area": "full", "num_sequences": "120"}

gen_array_tasks(
    distribute_func="projects.crispy_crispies.deployables.mpnn_binder",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="4G",
    nstruct=1,
    nstruct_per_task=1,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-266 /mnt/projects/crispy_shifty/projects/crispy_crispies/00_mpnn_binders/run.sh


In [5]:
!sbatch -a 1-266 /mnt/projects/crispy_shifty/projects/crispy_crispies/00_mpnn_binders/run.sh

Submitted batch job 33093502


### Collect scorefiles of the MPNN designed bound states and concatenate

In [3]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "00_mpnn_binders"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### Load resulting concatenated scorefile

In [4]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

### Dump scores_df as a CSV and then reload, for performance reasons

In [5]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Data exploration

In [6]:
print(len(scores_df))
print(list(scores_df.columns))

266
['bb_clash', 'binder_blocked_sap', 'buns_heavy_ball_1.1D', 'chA_length', 'chA_seq', 'closure_type', 'contact_molec_sq5_ap_target', 'contact_molec_sq5_apap_target', 'contact_molecular_surface', 'contact_molecular_surface_ap_target', 'contact_molecular_surface_apap_target', 'contact_patch', 'ddg', 'ddg_hydrophobic', 'disulfide_at', 'docked_helix', 'dslf_fa13', 'fa_atr', 'fa_atr_pocket', 'fa_dun_dev', 'fa_dun_rot', 'fa_dun_semi', 'fa_elec', 'fa_intra_atr_xover4', 'fa_intra_elec', 'fa_intra_rep_xover4', 'fa_intra_sol_xover4', 'fa_rep', 'fa_sol', 'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb', 'hxl_tors', 'hydrophobic_residue_contacts', 'interface_buried_sasa', 'interface_hydrophobic_count', 'interface_hydrophobics', 'interface_polar_count', 'lk_ball', 'lk_ball_bridge', 'lk_ball_bridge_uncpl', 'lk_ball_iso', 'mpnn_design_area', 'mpnn_seq_0000', 'mpnn_seq_0001', 'mpnn_seq_0002', 'mpnn_seq_0003', 'mpnn_seq_0004', 'mpnn_seq_0005', 'mpnn_seq_0006', 'mpnn_seq_0007', 'mpnn_seq_0008',

### Save individual fastas

In [7]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq")

  0%|          | 0/266 [00:00<?, ?it/s]

### Save a list of outputs
Sort by length

In [8]:
simulation_name = "00_mpnn_binders"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

scores_df = scores_df.sort_values("total_length")

with open(os.path.join(output_path, "mpnn_binders.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

  0%|          | 0/266 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [9]:
simulation_name = "00_mpnn_binders"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")

with open(os.path.join(output_path, "mpnn_binders.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/266 [00:00<?, ?it/s]

### Make seperate dfs of binders for the GIP/GLP/Glucagon family and for NPY

In [10]:
gip = scores_df.query("target_name == 'gip'")
glp = scores_df.query("target_name == 'glp'")
gluc = scores_df.query("target_name == 'glucagon'")
npy = scores_df.query("target_name == 'neuropeptideY'")

### Save a list of outputs
Sort by length  

In [11]:
simulation_name = "00_mpnn_binders"
output_path = os.path.join(os.getcwd(), f"projects/crispy_crispies/{simulation_name}")
with open(os.path.join(output_path, "gip.list"), "w") as f:
    for path in tqdm(gip.index):
        print(path, file=f)
with open(os.path.join(output_path, "glp.list"), "w") as f:
    for path in tqdm(glp.index):
        print(path, file=f)
with open(os.path.join(output_path, "gluc.list"), "w") as f:
    for path in tqdm(gluc.index):
        print(path, file=f)
with open(os.path.join(output_path, "npy.list"), "w") as f:
    for path in tqdm(npy.index):
        print(path, file=f)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

### Prototyping blocks